In [8]:
from scipy.sparse.linalg import bicg
import numpy as np
from numpy import linalg as lg
from scipy.io import mmread
from tabulate import tabulate
from time import time

A1=np.array(mmread('/content/bcsstk01.mtx.gz').todense())
#A1.shape=(48,46) Non symmetrique
A2=np.array(mmread('/content/impcol_c.mtx.gz').todense())
#A2.shape=(137,137) Non symmetrique
A3=np.array(mmread('/content/impcol_a.mtx.gz').todense())
#A3.shape=(207,207) Non symmetrique
A4=np.array(mmread('/content/mcca.mtx.gz').todense())
#A4.shape=(180, 180) Non symmetrique
A5=np.array(mmread('/content/bcsstm24.mtx.gz').todense())
#A5.shape=(3562,3562) Symmetrique definie positive
A6=np.array(mmread('/content/bcsstm19.mtx.gz').todense())
#A6.shape=(817,817) Symmetrique definie positive

B1=np.array(mmread('/content/bcsstk14.mtx.gz').todense())             
# A1.shape=(1806, 1806) real symmetric positive definite
B3=np.array(mmread('/content/bcsstm20.mtx.gz').todense())
# A3.shape=(485,485) real symmetric positive definite
B4=np.array(mmread('/content/bcsstm24.mtx.gz').todense())
# A4.shape=(3562,3562) real symmetric positive definite
B5=np.array(mmread('/content/bcsstm21.mtx.gz').todense())
B6=np.array(mmread('/content/bcsstk16.mtx.gz').todense())
B7=np.array(mmread('/content/bcsstk17.mtx.gz').todense())
B8=np.array(mmread('/content/bcsstk18.mtx.gz').todense())
B9=np.array(mmread('/content/bcsstm22.mtx.gz').todense())
B10=np.array(mmread('/content/bcsstk27.mtx.gz').todense())
B11=np.array(mmread('/content/bcsstm26.mtx.gz').todense())

C1=np.array(mmread('/content/west0067.mtx.gz').todense())    
C2=np.array(mmread('/content/jpwh_991.mtx.gz').todense())
C3=np.array(mmread('/content/impcol_a.mtx.gz').todense())
C4=np.array(mmread('/content/impcol_e.mtx.gz').todense())
C5=np.array(mmread('/content/sherman4.mtx.gz').todense())
C6=np.array(mmread('/content/orsreg_1.mtx.gz').todense())
C7=np.array(mmread('/content/orsirr_1.mtx.gz').todense())
C8=np.array(mmread('/content/add20.mtx.gz').todense())
C9=np.array(mmread('/content/gre__185.mtx.gz').todense())
C10=np.array(mmread('/content/fs_680_3.mtx.gz').todense())
C11=np.array(mmread('/content/fs_680_2.mtx.gz').todense())
C12=np.array(mmread('/content/west0381.mtx.gz').todense())
C13=np.array(mmread('/content/e05r0200.mtx.gz').todense())

#**BICG Method**

In [9]:
def bicg_iter(A, b,x0,maxiter,tol):
    num_iters = 0

    def callback(xk):
        nonlocal num_iters
        num_iters += 1
    return bicg(A, b, callback=callback,tol = tol,maxiter=maxiter)[0],num_iters

def BICG(A,b,x0,tol,maxiter):
  C=np.transpose(A)
  r0=b-np.dot(A,x0)
  rt0=r0/(lg.norm(r0)*lg.norm(r0))
  p0=r0
  pt0=rt0
  i=0
  while (i<maxiter):
    alpha=np.dot(r0,rt0)/np.dot(np.dot(A,p0),pt0)
    x1=x0+(alpha*p0)
    r1=r0-alpha*np.dot(A,p0)
    rt1=rt0-alpha*np.dot(C,pt0)
    beta=np.dot(r1,rt1)/np.dot(r0,rt0)
    p0=r1+beta*p0
    pt0=rt1+beta*pt0
    i=i+1
    r0=r1
    rt0=rt1
    x0=x1
    if lg.norm(r1)<tol:
      break
  return x1,i

# **Cluster 1:**

In [10]:
matricies=[B5,C1,B9,C2]
tol=1e-5
itermax = 10000

In [11]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  itersc = bicg_iter(matricies[i],b,x,itermax,tol)[1]  
  t3 = time()
  y=bicg(matricies[i],b,x,tol,itermax)[0] 
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms 
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1 = time()
  x,iterm= BICG(matricies[i], b, x, tol,itermax) 
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms  
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']  
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+--------------------+----------------+-----------+-------------+------------------+-----------+-------------+
| matricies    |   condition number |   Our_nbr_iter |      time |         res |   scipy_nbr_iter |   sc_time |      sc_res |
+==============+====================+================+===========+=============+==================+===========+=============+
| (3600, 3600) |            23.7154 |              3 | 121.666   | 8.60114e-12 |                3 |  83.6182  | 2.1765e-12  |
+--------------+--------------------+----------------+-----------+-------------+------------------+-----------+-------------+
| (67, 67)     |           130.217  |            133 |   7.98774 | 4.88637e-06 |              127 |  12.3634  | 6.95965e-05 |
+--------------+--------------------+----------------+-----------+-------------+------------------+-----------+-------------+
| (138, 138)   |           941.295  |             52 |   3.61943 | 7.82259e-06 |               45 |   5.03564 | 0.0001

# **Cluster 2:**

In [12]:
matricies=[C7,B10,A6,B11,C8,C9]
tol=1e-5
itermax = 10000

In [13]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  itersc = bicg_iter(matricies[i],b,x,itermax,tol)[1]  
  t3 = time()
  y=bicg(matricies[i],b,x,tol,itermax)[0] 
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms 
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1 = time()
  x,iterm= BICG(matricies[i], b, x, tol,itermax) 
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms  
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']  
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+--------------------+----------------+------------+-------------+------------------+-----------+-------------+
| matricies    |   condition number |   Our_nbr_iter |       time |         res |   scipy_nbr_iter |   sc_time |      sc_res |
+==============+====================+================+============+=============+==================+===========+=============+
| (1030, 1030) |            77142.8 |            978 |  1174.7    | 5.96212e-06 |              782 |   750.275 | 0.000207071 |
+--------------+--------------------+----------------+------------+-------------+------------------+-----------+-------------+
| (1224, 1224) |            24135.9 |           1005 |  1579.43   | 9.77683e-06 |              782 |   991.804 | 0.000346396 |
+--------------+--------------------+----------------+------------+-------------+------------------+-----------+-------------+
| (817, 817)   |           233734   |            544 |   417.995  | 5.99616e-06 |              393 |   251.322 

# **Cluster 3:**

In [14]:
matricies=[B6,B1,B4]
tol=1e-5
itermax = 100000

In [15]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  itersc = bicg_iter(matricies[i],b,x,itermax,tol)[1]  
  t3 = time()
  y=bicg(matricies[i],b,x,tol,itermax)[0] 
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms 
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1 = time()
  x,iterm= BICG(matricies[i], b, x, tol,itermax) 
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms  
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']  
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+--------------------+----------------+----------+-------------+------------------+-----------+-------------+
| matricies    |   condition number |   Our_nbr_iter |     time |         res |   scipy_nbr_iter |   sc_time |      sc_res |
+==============+====================+================+==========+=============+==================+===========+=============+
| (4884, 4884) |        4.94318e+09 |            484 |  24100.3 | 9.60472e-06 |              447 |   16598.2 | 0.000626704 |
+--------------+--------------------+----------------+----------+-------------+------------------+-----------+-------------+
| (1806, 1806) |        1.19232e+10 |          15368 | 101710   | 8.65357e-06 |            14038 |   67899.9 | 0.00032098  |
+--------------+--------------------+----------------+----------+-------------+------------------+-----------+-------------+
| (3562, 3562) |        1.80764e+13 |          30581 | 832081   | 8.31527e-06 |            18229 |  337808   | 0.000538547 |


# **Cluster 4:**

In [16]:
matricies=[C3,C4,C10,C12]
tol=1e-5
itermax = 10000

In [17]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  itersc = bicg_iter(matricies[i],b,x,itermax,tol)[1]  
  t3 = time()
  y=bicg(matricies[i],b,x,tol,itermax)[0] 
  t4 = time()
  sc_tmp = (t4-t3)*1000      # scipy time in ms 
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1 = time()
  x,iterm= BICG(matricies[i], b, x, tol,itermax) 
  t2 = time()
  tmp = (t2-t1)*1000      # Our time in ms  
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(tmp)
  l.append(res)
  l.append(itersc)
  l.append(sc_tmp)
  l.append(sc_res)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','time','res','scipy_nbr_iter','sc_time','sc_res']  
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+-------------+--------------------+----------------+----------+----------+------------------+-----------+-----------------+
| matricies   |   condition number |   Our_nbr_iter |     time |      res |   scipy_nbr_iter |   sc_time |          sc_res |
+=============+====================+================+==========+==========+==================+===========+=================+
| (207, 207)  |        1.35164e+08 |          10000 |  881.309 | 81452.8  |            10000 |   1250.99 |     8.51795e+53 |
+-------------+--------------------+----------------+----------+----------+------------------+-----------+-----------------+
| (225, 225)  |        7.10097e+06 |          10000 |  901.936 |  4749.63 |            10000 |   1218.78 | 10658.4         |
+-------------+--------------------+----------------+----------+----------+------------------+-----------+-----------------+
| (680, 680)  |        3.58695e+06 |          10000 | 4986.74  |  7981.02 |            10000 |   4382.54 | 31028.8         |
